### Youtube Sentence Adjust Word List Ratio

In [1]:
import pandas as pd
import numpy as np
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import glob

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
folder_name = "Turkish"
word_num = 28

#### Calculate Sentence Ratio By Word

In [4]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_adjust_word = df_word.iloc[0:word_num,]
df_adjust_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,ben,4908913
7,de,4880315
8,çok,4852169
9,ama,4661966


In [5]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [6]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(df_adjust_word["word"].to_list())
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [7]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id",f"{word_num}_word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"büyük, araçlar, kullanılıyor, kamyon, özgür, b...","daha, için"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"arka, burcu, bağlamak, tarafı, kolay, olmayacak",
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"tur, özgür, yarım, attıracağım, arabaya",
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"biraz, devam, et, burcu, tamam, oldu",daha
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"artarsa, burcu, kalınlığı, takacağız, iki, şim...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,25.1,"büyük, güçlü, ellerde, herkes, yanlış, ki, uma...","bu, ve, için"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"halde, herkesin, kurumlarda, gelişir, eşit, öz...","daha, bir, ve"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"belirttim, kaynakları, olarak, yararlandığım, ...",


In [8]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"büyük, araçlar, kullanılıyor, kamyon, özgür, b...","daha, için"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"arka, burcu, bağlamak, tarafı, kolay, olmayacak",NaN
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"tur, özgür, yarım, attıracağım, arabaya",NaN
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"biraz, devam, et, burcu, tamam, oldu",daha
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"artarsa, burcu, kalınlığı, takacağız, iki, şim...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,25.1,"büyük, güçlü, ellerde, herkes, yanlış, ki, uma...","bu, ve, için"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"halde, herkesin, kurumlarda, gelişir, eşit, öz...","daha, bir, ve"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"belirttim, kaynakları, olarak, yararlandığım, ...",NaN


In [9]:
df_sentence_ratio_result.to_csv(f"Youtube_Sentence_{word_num}_Word_Ratio_Result.csv", index=False)

#### Calculate Videoid Ratio By Word

In [10]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_adjust_word = df_word.iloc[0:word_num,]
df_adjust_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,ben,4908913
7,de,4880315
8,çok,4852169
9,ama,4661966


In [11]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [12]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence

,video_id,sentence
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...
4,00fr8yyBQYo,bill gates and george soros developed a new co...
...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...


In [13]:
def videoid_word_ratio(df_word, df_videoid_sentence, word_column_name, videoid_sentence_column_name, word_num):
    word_set = set(df_word[f"{word_column_name}"].to_list())
    for i in range(len(df_videoid_sentence)):
        text = df_videoid_sentence.loc[i,f"{videoid_sentence_column_name}"]
        text_word = re.findall(r"\w+", text, re.UNICODE)
        text_word_set = set(text_word)
        intersect_word = list(word_set.intersection(text_word_set))
        different_word = list(text_word_set.difference(word_set))
        df_videoid_sentence.loc[i,f"{word_num}_word_ratio"] = round(((len(intersect_word)/len(text_word)+0.001)*100),)
        df_videoid_sentence.loc[i,"different_word"] = ", ".join(different_word)
        df_videoid_sentence.loc[i,"intersect_word"] = ", ".join(intersect_word)
        
    return df_videoid_sentence

In [14]:
df_text_ratio_result = videoid_word_ratio(df_adjust_word, df_videoid_sentence, "word", "sentence", word_num)
df_text_ratio_result

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,7.0,"şöyle, karadeniz, eridikten, kıvamında, ayarı,...","da, ama, var, ve, için, kadar, çok, bu, de, gi..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,2.0,"memnunlar, demokrasiye, beri, basın, batırdık,...","yok, ama, mı, çok, hayır, daha, bir, da, evet,..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,11.0,"isen, ölürümi, dur, bui, geçirsini, ievlat, io...","sen, ben, ne, de, mi"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,14.0,"durumunuzda, gece, nanın, abla, yabancı, bilmi...","beni, sen, da, ama, var, onu, ne, kadar, çok, ..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.0,"twilight, something, ground, today, risk, acce...",
...,...,...,...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,3.0,"pazardan, karadeniz, ederimah, geldin, yoruldu...","beni, iyi, gibi, da, ama, ben, var, ne, ve, iç..."
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,15.0,"yaklaşırsın, elimden, uzaklaş, dışında, kızını...","da, ben, bu, de, gibi, seni, mi, bana"
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,1.0,"şöyle, batımını, satacağız, kumsal, odasından,...","yok, ama, mı, çok, daha, bir, beni, da, evet, ..."
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,14.0,"videolara, büyük, reklamlardan, işler, vardı, ...","ben, ve, için, kadar, çok, bir"


In [15]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,7.0,"şöyle, karadeniz, eridikten, kıvamında, ayarı,...","da, ama, var, ve, için, kadar, çok, bu, de, gi..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,2.0,"memnunlar, demokrasiye, beri, basın, batırdık,...","yok, ama, mı, çok, hayır, daha, bir, da, evet,..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,11.0,"isen, ölürümi, dur, bui, geçirsini, ievlat, io...","sen, ben, ne, de, mi"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,14.0,"durumunuzda, gece, nanın, abla, yabancı, bilmi...","beni, sen, da, ama, var, onu, ne, kadar, çok, ..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.0,"twilight, something, ground, today, risk, acce...",NaN
...,...,...,...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,3.0,"pazardan, karadeniz, ederimah, geldin, yoruldu...","beni, iyi, gibi, da, ama, ben, var, ne, ve, iç..."
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,15.0,"yaklaşırsın, elimden, uzaklaş, dışında, kızını...","da, ben, bu, de, gibi, seni, mi, bana"
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,1.0,"şöyle, batımını, satacağız, kumsal, odasından,...","yok, ama, mı, çok, daha, bir, beni, da, evet, ..."
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,14.0,"videolara, büyük, reklamlardan, işler, vardı, ...","ben, ve, için, kadar, çok, bir"


In [16]:
df_text_ratio_result.to_csv(f"Youtube_Videoid_Text_{word_num}_Word_Ratio_Result.csv", index=False)

#### Select Adjust Sentence Ratio Result

In [28]:
df_sentence_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_Ratio_Result.csv")
#df_sentence_ratio_result = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_Ratio_Result.csv")
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"büyük, araçlar, kullanılıyor, kamyon, özgür, b...","daha, için"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"arka, burcu, bağlamak, tarafı, kolay, olmayacak",NaN
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"tur, özgür, yarım, attıracağım, arabaya",NaN
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"biraz, devam, et, burcu, tamam, oldu",daha
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"artarsa, burcu, kalınlığı, takacağız, iki, şim...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,25.1,"büyük, güçlü, ellerde, herkes, yanlış, ki, uma...","bu, ve, için"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"halde, herkesin, kurumlarda, gelişir, eşit, öz...","daha, bir, ve"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"belirttim, kaynakları, olarak, yararlandığım, ...",NaN


In [29]:
adjust_word_ratio = 100
df_adjust_ratio = df_sentence_ratio_result[df_sentence_ratio_result[f"{word_num}_word_ratio"] >= adjust_word_ratio]
df_adjust_ratio.reset_index(inplace=True, drop=True)
df_adjust_ratio

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,222,00:27:46.048,00:27:46.560,evet,8V9tq1pe8eI,100.1,NaN,evet
1,605,00:10:35.360,00:10:36.520,evet,cQjX9DzXTw0,100.1,NaN,evet
2,1906,00:01:25.760,00:01:26.528,gibi,owluCDXgJSE,100.1,NaN,gibi
3,1923,00:02:02.368,00:02:02.880,ve,owluCDXgJSE,100.1,NaN,ve
4,2018,00:05:33.823,00:05:37.151,ve,owluCDXgJSE,100.1,NaN,ve
...,...,...,...,...,...,...,...,...
32045,3036709,00:17:25.970,00:17:26.000,bu,sSTIfQUm18Q,100.1,NaN,bu
32046,3036732,00:20:21.720,00:20:21.780,ben,sSTIfQUm18Q,100.1,NaN,ben
32047,3036760,00:25:30.370,00:25:30.400,bu,sSTIfQUm18Q,100.1,NaN,bu
32048,3036762,00:25:54.640,00:25:54.700,sen,sSTIfQUm18Q,100.1,NaN,sen


In [19]:
df_adjust_ratio.to_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.csv", index=False)

In [20]:
df_adjust_ratio.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.xlsx", index=False)

#### Calculate Sequential Sentence Length Of Adjust Ratio Result

In [32]:
df_sentence_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_Ratio_Result.csv")
#df_sentence_ratio_result = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_Ratio_Result.csv")
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"büyük, araçlar, kullanılıyor, kamyon, özgür, b...","daha, için"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"arka, burcu, bağlamak, tarafı, kolay, olmayacak",NaN
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"tur, özgür, yarım, attıracağım, arabaya",NaN
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"biraz, devam, et, burcu, tamam, oldu",daha
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"artarsa, burcu, kalınlığı, takacağız, iki, şim...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,25.1,"büyük, güçlü, ellerde, herkes, yanlış, ki, uma...","bu, ve, için"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"halde, herkesin, kurumlarda, gelişir, eşit, öz...","daha, bir, ve"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"belirttim, kaynakları, olarak, yararlandığım, ...",NaN


In [ ]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].astype("object")
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].astype("object")

In [33]:
df_adjust_ratio = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.csv")
#df_adjust_ratio = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,222,00:27:46.048,00:27:46.560,evet,8V9tq1pe8eI,100.1,NaN,evet
1,605,00:10:35.360,00:10:36.520,evet,cQjX9DzXTw0,100.1,NaN,evet
2,1906,00:01:25.760,00:01:26.528,gibi,owluCDXgJSE,100.1,NaN,gibi
3,1923,00:02:02.368,00:02:02.880,ve,owluCDXgJSE,100.1,NaN,ve
4,2018,00:05:33.823,00:05:37.151,ve,owluCDXgJSE,100.1,NaN,ve
...,...,...,...,...,...,...,...,...
32045,3036709,00:17:25.970,00:17:26.000,bu,sSTIfQUm18Q,100.1,NaN,bu
32046,3036732,00:20:21.720,00:20:21.780,ben,sSTIfQUm18Q,100.1,NaN,ben
32047,3036760,00:25:30.370,00:25:30.400,bu,sSTIfQUm18Q,100.1,NaN,bu
32048,3036762,00:25:54.640,00:25:54.700,sen,sSTIfQUm18Q,100.1,NaN,sen


In [34]:
df_adjust_ratio["different_word"] = df_adjust_ratio["different_word"].astype("object")
df_adjust_ratio["intersect_word"] = df_adjust_ratio["intersect_word"].astype("object")

In [35]:
df_adjust_ratio["with_word"] = 1
df_adjust_ratio

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word,with_word
0,222,00:27:46.048,00:27:46.560,evet,8V9tq1pe8eI,100.1,NaN,evet,1
1,605,00:10:35.360,00:10:36.520,evet,cQjX9DzXTw0,100.1,NaN,evet,1
2,1906,00:01:25.760,00:01:26.528,gibi,owluCDXgJSE,100.1,NaN,gibi,1
3,1923,00:02:02.368,00:02:02.880,ve,owluCDXgJSE,100.1,NaN,ve,1
4,2018,00:05:33.823,00:05:37.151,ve,owluCDXgJSE,100.1,NaN,ve,1
...,...,...,...,...,...,...,...,...,...
32045,3036709,00:17:25.970,00:17:26.000,bu,sSTIfQUm18Q,100.1,NaN,bu,1
32046,3036732,00:20:21.720,00:20:21.780,ben,sSTIfQUm18Q,100.1,NaN,ben,1
32047,3036760,00:25:30.370,00:25:30.400,bu,sSTIfQUm18Q,100.1,NaN,bu,1
32048,3036762,00:25:54.640,00:25:54.700,sen,sSTIfQUm18Q,100.1,NaN,sen,1


In [36]:
df_adjust_ratio_flag = pd.merge(df_sentence_ratio_result , df_adjust_ratio, how="left", on=["index","sentence","start_time","end_time","video_id",f"{word_num}_word_ratio","different_word","intersect_word"])
df_adjust_ratio_flag["with_word"].fillna(0, inplace=True)
df_adjust_ratio_flag

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word,with_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"büyük, araçlar, kullanılıyor, kamyon, özgür, b...","daha, için",0.0
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"arka, burcu, bağlamak, tarafı, kolay, olmayacak",NaN,0.0
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"tur, özgür, yarım, attıracağım, arabaya",NaN,0.0
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"biraz, devam, et, burcu, tamam, oldu",daha,0.0
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"artarsa, burcu, kalınlığı, takacağız, iki, şim...",ama,0.0
...,...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,25.1,"büyük, güçlü, ellerde, herkes, yanlış, ki, uma...","bu, ve, için",0.0
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"halde, herkesin, kurumlarda, gelişir, eşit, öz...","daha, bir, ve",0.0
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için,0.0
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"belirttim, kaynakları, olarak, yararlandığım, ...",NaN,0.0


In [37]:
df_adjust_ratio_flag.with_word.value_counts()

0.0    3004887
1.0      32050
Name: with_word, dtype: int64

In [38]:
# Code Running For Video_id Sentence
zeros = []
ones = []
zero_list = []
one_list = []
for i in range(len(df_adjust_ratio_flag)):
    videoid = df_adjust_ratio_flag.loc[i,"video_id"]    
    if df_adjust_ratio_flag.loc[i,"with_word"] == 0:
        zeros.append(i)
        try:
            one_list.append((ones[0],len(ones)))
        except:
            pass
        ones = []
    else:
        ones.append(i)
        try:
            zero_list.append((zeros[0],len(zeros)))
        except:
            pass
        zeros = []

    try:
        if df_adjust_ratio_flag.loc[i+1,"video_id"] != videoid:
            try:
                one_list.append((ones[0],len(ones)))
                ones = []
            except:
                pass
            try:
                zero_list.append((zeros[0],len(zeros)))
                zeros = []
            except:
                pass
        else:
            pass
    except:
        pass

try:
    one_list.append((ones[0],len(ones)))
except:
    pass
try:
    zero_list.append((zeros[0],len(zeros)))
except:
    pass

In [24]:
df_one = pd.DataFrame(one_list)
df_one.rename(columns={0:"index",1:"length"}, inplace=True)
df_one.drop_duplicates(inplace=True)
df_one.sort_values(by="length", ascending=False, inplace=True)
df_one.reset_index(drop=True, inplace=True)
df_one

,index,length
0,168924,9
1,16558,8
2,168934,7
3,9734,5
4,9787,4
...,...,...
8969,111941,1
8970,111924,1
8971,111921,1
8972,111714,1


In [25]:
df_one_all = pd.DataFrame()
for i in range(len(df_one)):
    index = df_one.loc[i,"index"]
    length = df_one.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_all = pd.concat([df_one_all, df_video],axis=0)
df_one_all.reset_index(inplace=True, drop=True)
df_one_all

,index,start_time,end_time,sentence,video_id,206_word_ratio,different_word,intersect_word,with_word
0,168924,00:00:00.460,00:00:01.400,baba,nnYAt9HWiY4,100.1,NaN,baba,1.0
1,168925,00:00:07.420,00:00:09.020,baba,nnYAt9HWiY4,100.1,NaN,baba,1.0
2,168926,00:00:09.140,00:00:09.920,efendim,nnYAt9HWiY4,100.1,NaN,efendim,1.0
3,168927,00:00:09.920,00:00:10.720,baba,nnYAt9HWiY4,100.1,NaN,baba,1.0
4,168928,00:00:10.720,00:00:11.620,efendim,nnYAt9HWiY4,100.1,NaN,efendim,1.0
...,...,...,...,...,...,...,...,...,...
9698,111941,00:07:00.700,00:07:06.267,ama burada,Ao3UnRqDeVg,100.1,NaN,"burada, ama",1.0
9699,111924,00:04:52.334,00:05:02.267,şimdi,Ao3UnRqDeVg,100.1,NaN,şimdi,1.0
9700,111921,00:04:31.900,00:04:37.100,evet hala,Ao3UnRqDeVg,100.1,NaN,"evet, hala",1.0
9701,111714,00:10:53.840,00:10:53.900,çok,9EaLkRsP3Ps,100.1,NaN,çok,1.0


In [26]:
#df_one_all.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sequential_Sentences_And_Others.xlsx", index=False)

In [27]:
# Only Sequential Sentences
df_one_sequential = df_one[df_one.length > 1]
df_one_sequential

,index,length
0,168924,9
1,16558,8
2,168934,7
3,9734,5
4,9787,4
...,...,...
626,11803,2
627,298742,2
628,188971,2
629,35245,2


In [28]:
df_one_sequential_all = pd.DataFrame()
for i in range(len(df_one_sequential)):
    index = df_one_sequential.loc[i,"index"]
    length = df_one_sequential.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_sequential_all = pd.concat([df_one_sequential_all, df_video],axis=0)
df_one_sequential_all.reset_index(inplace=True, drop=True)
df_one_sequential_all

,index,start_time,end_time,sentence,video_id,206_word_ratio,different_word,intersect_word,with_word
0,168924,00:00:00.460,00:00:01.400,baba,nnYAt9HWiY4,100.1,NaN,baba,1.0
1,168925,00:00:07.420,00:00:09.020,baba,nnYAt9HWiY4,100.1,NaN,baba,1.0
2,168926,00:00:09.140,00:00:09.920,efendim,nnYAt9HWiY4,100.1,NaN,efendim,1.0
3,168927,00:00:09.920,00:00:10.720,baba,nnYAt9HWiY4,100.1,NaN,baba,1.0
4,168928,00:00:10.720,00:00:11.620,efendim,nnYAt9HWiY4,100.1,NaN,efendim,1.0
...,...,...,...,...,...,...,...,...,...
1355,188972,00:17:35.487,00:17:36.511,zaman,rXAAGktXqQ4,100.1,NaN,zaman,1.0
1356,35245,00:03:32.164,00:03:36.700,yani,dD3A1IseY00,100.1,NaN,yani,1.0
1357,35246,00:03:39.960,00:03:43.380,ama,dD3A1IseY00,100.1,NaN,ama,1.0
1358,281795,00:21:58.144,00:21:58.656,tamam mı,xHyS_6Hq5GA,100.1,NaN,"mı, tamam",1.0


In [29]:
#df_one_sequential_all.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Only_Sequential_Sentences.xlsx", index=False)

#### Count Sentence Words By Adjust Column

In [30]:
#df_one_sequential_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/{level.lower().capitalize()}/4-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Only_Sequential_Sentences.xlsx")
df_one_sequential_all = pd.read_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Only_Sequential_Sentences.xlsx")
df_one_sequential_all

,index,start_time,end_time,sentence,video_id,206_word_ratio,different_word,intersect_word,with_word
0,168924,00:00:00.460,00:00:01.400,baba,nnYAt9HWiY4,100.1,NaN,baba,1
1,168925,00:00:07.420,00:00:09.020,baba,nnYAt9HWiY4,100.1,NaN,baba,1
2,168926,00:00:09.140,00:00:09.920,efendim,nnYAt9HWiY4,100.1,NaN,efendim,1
3,168927,00:00:09.920,00:00:10.720,baba,nnYAt9HWiY4,100.1,NaN,baba,1
4,168928,00:00:10.720,00:00:11.620,efendim,nnYAt9HWiY4,100.1,NaN,efendim,1
...,...,...,...,...,...,...,...,...,...
1355,188972,00:17:35.487,00:17:36.511,zaman,rXAAGktXqQ4,100.1,NaN,zaman,1
1356,35245,00:03:32.164,00:03:36.700,yani,dD3A1IseY00,100.1,NaN,yani,1
1357,35246,00:03:39.960,00:03:43.380,ama,dD3A1IseY00,100.1,NaN,ama,1
1358,281795,00:21:58.144,00:21:58.656,tamam mı,xHyS_6Hq5GA,100.1,NaN,"mı, tamam",1


In [31]:
def text_to_list_df(df_target, target_column): # df_target include dict value columns, target_column occurs dict
    text_value_list = []
    for i in df_target[f"{target_column}"]:
        try:
            var = re.findall(r"\w+", i)
            for j in var:
                text_value_list.append(j)
        except:
            pass
    df_alone_value = pd.DataFrame(text_value_list)
    df_alone_value.rename(columns={0:f"{target_column}_alone"}, inplace=True)
    df_unique_count = pd.DataFrame(df_alone_value.value_counts(ascending=False))
    df_unique_count.reset_index(inplace=True)
    df_unique_count.rename(columns={0:"count"}, inplace=True)
            
    return df_unique_count

In [32]:
df_unique_word_count = text_to_list_df(df_one_sequential_all, "intersect_word")
df_unique_word_count

,intersect_word_alone,count
0,bu,114
1,evet,101
2,tamam,91
3,bir,64
4,ne,61
...,...,...
171,hiç,1
172,değilim,1
173,seninle,1
174,yardım,1


In [33]:
#df_unique_word_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sequential_Sentence_Words_Count.xlsx", index=False)

#### Count Adjust Sentence Ratio Result By Videoid

In [34]:
df_ratio_count = pd.DataFrame(df_adjust_ratio.groupby("video_id")[f"{word_num}_word_ratio"].count())
df_ratio_count.reset_index(inplace=True)
df_ratio_count.rename(columns={f"{word_num}_word_ratio":f"{word_num}_word_ratio_count"}, inplace=True)
df_ratio_count.sort_values(by=f"{word_num}_word_ratio_count", ascending=False, inplace=True)
df_ratio_count.reset_index(drop=True,inplace=True)
df_ratio_count

,video_id,206_word_ratio_count
0,xHyS_6Hq5GA,120
1,eA6FwMDpvfs,89
2,8PfssuZA6GM,88
3,UDDfBEUGY60,78
4,Aii8sJ5Uw_8,74
...,...,...
1811,6TgQWjyn_6k,1
1812,XyJzGMsxGTI,1
1813,Xzn8PovqDcU,1
1814,Y26YgIFEQcw,1


In [35]:
#df_ratio_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Videoid_Count.xlsx", index=False)

#### Find Word In Sentence Time Interval

In [5]:
df_word_count_low = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/{level.lower().capitalize()}/3-Lesson Level Result/Lesson_{level.lower().capitalize()}_Low_Count_Words.xlsx")
df_word_count_low

,word,count
0,yardım,6
1,ve,6
2,oh,6
3,ona,6
4,bize,6
...,...,...
116,vardı,1
117,size,1
118,tüm,1
119,gece,1


In [6]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM
1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM
2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM
3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM
4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM
...,...,...,...,...
387903,00:02:11.720,00:02:14.440,gözlerin bir an bile çıkmadı aklımdan,7R3K6EDaBGE
387904,00:02:15.560,00:02:19.320,koynumdaysa hala o ufak bin lira,7R3K6EDaBGE
387905,00:02:20.320,00:02:21.200,sahi,7R3K6EDaBGE
387906,00:02:21.720,00:02:24.480,diğer eşi sende mi hala,7R3K6EDaBGE


In [7]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [8]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,6.68,7.34,sen kimsin peki,JR7nKXxUVtM
1,7.34,8.60,parolayı söyle,JR7nKXxUVtM
2,9.44,11.06,ya ben kim olacağım,JR7nKXxUVtM
3,11.06,11.86,keloğlan ım,JR7nKXxUVtM
4,13.38,13.88,ey,JR7nKXxUVtM
...,...,...,...,...
387903,131.72,134.44,gözlerin bir an bile çıkmadı aklımdan,7R3K6EDaBGE
387904,135.56,139.32,koynumdaysa hala o ufak bin lira,7R3K6EDaBGE
387905,140.32,141.20,sahi,7R3K6EDaBGE
387906,141.72,144.48,diğer eşi sende mi hala,7R3K6EDaBGE


In [9]:
word_list = df_word_count_low.iloc[:,0].values.tolist()

In [10]:
df_word_result = pd.DataFrame()
for i in word_list:
    try:
        word_in_video = df_youtube_sentence[df_youtube_sentence.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].sample(1)
    except:
        word_in_video = df_youtube_sentence[df_youtube_sentence.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(1) 
    word_in_video.insert(0,"word",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result = df_word_result.sort_values(by=["video_id","start_time"], ascending=True)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result

,word,start_time,end_time,sentence,video_id
0,bence,226.480,227.680,bence bu daha kötü,0VppXfycGc8
1,kadar,264.900,277.350,mutluyum da bir o kadar bu şöyle dileklerimiz ...,0zfPvkfce4w
2,küçük,63.370,72.340,evet şu an ekran kartı o çok küçük ya bu ne ka...,0zpbbhfIR9A
3,aslında,348.759,356.949,ben gerçekte aslında,1boYNWzDEVQ
4,biri,155.637,159.310,benim bu evde en beğendiğim özelliklerden biri...,28fIr7Di0mk
...,...,...,...,...,...
116,dakika,209.880,230.850,bu gene şimdi tamam bir dakika bir durum anali...,xlexPqSswQc
117,diye,147.300,152.310,böyle lavanta bahçesi oraya gidelim işte hemen...,xspshzt0uCo
118,onun,594.487,596.487,onun iradesine bağlanırlar,y2A6i8ZdnY4
119,başka,269.680,277.820,çok pahalandı ve hemen arkasından gelen ürünüm...,yXl7G4tvyN0


In [11]:
word_time_loc_list = []
def word_time_loc(df):
    for i in range(len(df)):
        word = df.loc[i,"word"]
        start_time = df.loc[i,"start_time"]
        end_time = df.loc[i,"end_time"]
        sentence = df.loc[i,"sentence"]
        video_id = df.loc[i,"video_id"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=["word","start_time","end_time","sentence","video_id"])

    return df_word_time_loc

In [12]:
df_word_time_loc_result = word_time_loc(df_word_result)
df_word_time_loc_result

,word,start_time,end_time,sentence,video_id
0,bence,226.480000,226.880000,bence bu daha kötü,0VppXfycGc8
1,kadar,267.151596,268.078723,mutluyum da bir o kadar bu şöyle dileklerimiz ...,0zfPvkfce4w
2,küçük,66.470741,67.245926,evet şu an ekran kartı o çok küçük ya bu ne ka...,0zpbbhfIR9A
3,aslında,353.673000,356.949000,ben gerçekte aslında,1boYNWzDEVQ
4,biri,158.425759,158.833870,benim bu evde en beğendiğim özelliklerden biri...,28fIr7Di0mk
...,...,...,...,...,...
116,dakika,214.904063,216.651562,bu gene şimdi tamam bir dakika bir durum anali...,xlexPqSswQc
117,diye,151.271341,151.637927,böyle lavanta bahçesi oraya gidelim işte hemen...,xspshzt0uCo
118,onun,594.487000,594.871615,onun iradesine bağlanırlar,y2A6i8ZdnY4
119,başka,276.141649,276.729072,çok pahalandı ve hemen arkasından gelen ürünüm...,yXl7G4tvyN0


In [ ]:
time_shift = 0.3
df_word_time_loc_result.start_time = df_word_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_time_loc_result.end_time = df_word_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_time_loc_result

In [ ]:
#df_word_time_loc_result.to_excel(f"Youtube_Sentence_{word_num}_Low_Word_Time_Interval.xlsx", index=False)